In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import cv2

# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Some modules to display an animation using imageio.
import imageio
from IPython.display import HTML, display
import pandas as pd

In [4]:
# Import TF and TF Hub libraries.
import tensorflow as tf
import tensorflow_hub as hub

# Load the input image.
image_path = r'D:\Pemrograman\Python\Project\Pose Estimation\poses_images_out_train\adho mukha svanasana\1. 1.png'
image = tf.io.read_file(image_path)
image = tf.compat.v1.image.decode_jpeg(image)[:,:,:3]
image = tf.expand_dims(image, axis=0)
# Resize and pad the image to keep the aspect ratio and fit the expected size.
image = tf.cast(tf.image.resize_with_pad(image, 192, 192), dtype=tf.int32)

# Download the model from TF Hub.
model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = model.signatures['serving_default']

# Run model inference.
outputs = movenet(image)
# Output is a [1, 1, 17, 3] tensor.
keypoints = outputs['output_0']


In [5]:
# 17 keypoints in the model output
kp_descriptions = [
    'nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear',
    'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow',
    'left_wrist', 'right_wrist', 'left_hip', 'right_hip',
    'left_knee', 'right_knee', 'left_ankle', 'right_ankle'
]
# Rename columns in the DataFrames according to the values
columns = []
for point in kp_descriptions:
    for value in ('y', 'x', 'score'):
        columns.append(f'{point}_{value}')

In [6]:
pd.DataFrame(np.array(keypoints[0][0]).flatten().reshape(1,-1),columns = columns)

,nose_y,nose_x,nose_score,left_eye_y,left_eye_x,left_eye_score,right_eye_y,right_eye_x,right_eye_score,left_ear_y,...,left_knee_score,right_knee_y,right_knee_x,right_knee_score,left_ankle_y,left_ankle_x,left_ankle_score,right_ankle_y,right_ankle_x,right_ankle_score
0,0.580765,0.427303,0.675039,0.579229,0.412077,0.47334,0.58186,0.412161,0.524445,0.544553,...,0.677413,0.497728,0.70222,0.804713,0.646824,0.821644,0.755961,0.623078,0.783787,0.604527


In [11]:
# import required libraries
import numpy as np
from numpy.linalg import norm
 
# define two lists or array
A = np.array([0.58076525, 0.42730328, 0.67503864, 0.5792289 , 0.41207743,
       0.47333997, 0.58185965, 0.41216052, 0.52444524, 0.5445528 ,
       0.39484406, 0.45617175, 0.5446289 , 0.3936121 , 0.46745512,
       0.5016635 , 0.4046818 , 0.7063899 , 0.49563456, 0.40258092,
       0.54718405, 0.60904396, 0.34574008, 0.7440503 , 0.5912181 ,
       0.34356624, 0.6541515 , 0.70214546, 0.26372153, 0.9110583 ,
       0.6600009 , 0.27307063, 0.38330933, 0.32035175, 0.6203919 ,
       0.61942005, 0.32266206, 0.60961163, 0.76251876, 0.4952395 ,
       0.7256286 , 0.6774133 , 0.4977281 , 0.70222014, 0.80471295,
       0.6468238 , 0.8216442 , 0.75596136, 0.62307787, 0.78378725,
       0.60452706])

B = np.array([0.5749743 , 0.42486113, 0.54318196, 0.57792485, 0.40930325,
       0.5496756 , 0.5782155 , 0.40489376, 0.57750416, 0.5340253 ,
       0.38575894, 0.5966261 , 0.5361736 , 0.37668896, 0.54088706,
       0.50579745, 0.43160397, 0.6138604 , 0.48600656, 0.37216204,
       0.6575917 , 0.63704824, 0.33847758, 0.64200926, 0.587188  ,
       0.31143132, 0.7014244 , 0.72440684, 0.2353007 , 0.6007997 ,
       0.6784536 , 0.23021728, 0.51602674, 0.30333444, 0.6349661 ,
       0.74985135, 0.31460917, 0.60921407, 0.68592465, 0.47325823,
       0.77171266, 0.6738131 , 0.48240095, 0.73322797, 0.8058868 ,
       0.6358363 , 0.84106433, 0.6906332 , 0.6143594 , 0.7967786 ,
       0.7987496 ])
 
print("A:", A)
print("B:", B)
 
# compute cosine similarity
cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

A: [0.58076525 0.42730328 0.67503864 0.5792289  0.41207743 0.47333997
 0.58185965 0.41216052 0.52444524 0.5445528  0.39484406 0.45617175
 0.5446289  0.3936121  0.46745512 0.5016635  0.4046818  0.7063899
 0.49563456 0.40258092 0.54718405 0.60904396 0.34574008 0.7440503
 0.5912181  0.34356624 0.6541515  0.70214546 0.26372153 0.9110583
 0.6600009  0.27307063 0.38330933 0.32035175 0.6203919  0.61942005
 0.32266206 0.60961163 0.76251876 0.4952395  0.7256286  0.6774133
 0.4977281  0.70222014 0.80471295 0.6468238  0.8216442  0.75596136
 0.62307787 0.78378725 0.60452706]
B: [0.5749743  0.42486113 0.54318196 0.57792485 0.40930325 0.5496756
 0.5782155  0.40489376 0.57750416 0.5340253  0.38575894 0.5966261
 0.5361736  0.37668896 0.54088706 0.50579745 0.43160397 0.6138604
 0.48600656 0.37216204 0.6575917  0.63704824 0.33847758 0.64200926
 0.587188   0.31143132 0.7014244  0.72440684 0.2353007  0.6007997
 0.6784536  0.23021728 0.51602674 0.30333444 0.6349661  0.74985135
 0.31460917 0.60921407 0.6859

In [7]:
np.array(keypoints[0][0]).flatten().reshape(1,-1)

array([[0.58076525, 0.42730328, 0.67503864, 0.5792289 , 0.41207743,
        0.47333997, 0.58185965, 0.41216052, 0.52444524, 0.5445528 ,
        0.39484406, 0.45617175, 0.5446289 , 0.3936121 , 0.46745512,
        0.5016635 , 0.4046818 , 0.7063899 , 0.49563456, 0.40258092,
        0.54718405, 0.60904396, 0.34574008, 0.7440503 , 0.5912181 ,
        0.34356624, 0.6541515 , 0.70214546, 0.26372153, 0.9110583 ,
        0.6600009 , 0.27307063, 0.38330933, 0.32035175, 0.6203919 ,
        0.61942005, 0.32266206, 0.60961163, 0.76251876, 0.4952395 ,
        0.7256286 , 0.6774133 , 0.4977281 , 0.70222014, 0.80471295,
        0.6468238 , 0.8216442 , 0.75596136, 0.62307787, 0.78378725,
        0.60452706]], dtype=float32)